In [20]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

In [42]:
from sqlalchemy import create_engine
def fetch_data_from_sql():
    # Construct the connection string
    connection_str = "mssql+pyodbc://fibs_pc/PortfolioProject_MarketingAnalytics?driver=SQL+Server&trusted_connection=yes"
    
    # Create the SQLAlchemy engine
    engine = create_engine(connection_str)
    
    # Define your query
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"
    
    # Execute the query and read the data into a pandas DataFrame
    df = pd.read_sql_query(query, engine)
    
    # Close the connection (SQLAlchemy handles this automatically)
    return df

In [34]:
'''
def fetch_data_from_sql():
    connection_str = (
        "Driver={SQL Server};"
        "Server=fibs_pc;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    connection = pyodbc.connect(connection_str)
    query = "select ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText from dbo.customer_reviews"
    
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df
'''

In [43]:
reviews_df = fetch_data_from_sql()
sia = SentimentIntensityAnalyzer()

In [44]:
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [52]:
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'positive'
        elif rating == 3:
            return 'mixed positve'
        else:
            return 'mixed negative'
    elif score < -0.05:
        if rating <=2:
            return 'negative'
        elif rating == 3:
          return 'mixed negative'
        else:
            return 'mixed positive'
    else:
        if rating >= 4:
            return 'positive'
        elif rating <= 2:
            return 'negative'
        else:
            return 'neutral'


In [54]:
def sentiment_bucket(score):
    if score >= 0.05:
        return '0.5 to 1.0' #strong positive
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  #mild positive
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'   #mild negative
    else:
        return '-1.0 to -0.5'   #strong negative

In [55]:
#apply sentiment analysis to calculate sent scores and reviews
reviews_df['SentimentScore'] = reviews_df['ReviewText'].apply(calculate_sentiment)

#applying sent category using text and ratings
reviews_df['SentimentCategory'] = reviews_df.apply(lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

#define ranges using sent bucket
reviews_df['SentimentBucket'] = reviews_df['SentimentScore'].apply(sentiment_bucket)

In [56]:
reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,mixed negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,mixed positve,0.5 to 1.0
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,mixed negative,-0.49 to 0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,mixed negative,-0.49 to 0.0
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,negative,-0.49 to 0.0
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,positive,0.5 to 1.0
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,negative,0.0 to 0.49


In [58]:
reviews_df.to_csv('fact_customers_reviews_with_sentiments.csv', index=False)